Code based on this fantastic notebook from [Thomas Capelle (wandb)](https://github.com/tcapelle/mistral_wandb/blob/main/mistral_wandb.ipynb).

In [10]:
%load_ext autoreload
%autoreload 2

import weave
from loguru import logger
import pandas as pd
import random
from mistralai.client import MistralClient
from mistralai.models.jobs import WandbIntegrationIn, TrainingParameters

from mistral_fine_tuning.utils import read_fine_tuning_file
from mistral_fine_tuning.reformat import reformat_jsonl

from dotenv import load_dotenv
load_dotenv()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [5]:
df = pd.read_json('../data/processed/jokes.jsonl', lines=True)

In [6]:
df.head()

,text,keyword
0,"Oye, fíjate que llega un indígena al médico y ...",indígena
1,Llega un hombre y le dice al doctor: 'Me acabo...,luna de miel
2,Un hombre llega a una tienda de deportes compl...,desnudo
3,"Y llega Caperucita Roja a ver a la abuelita. ""...",Caperucita Roja
4,"La situación, la crisis inicial, no tiene más....",crisis


In [7]:
df_train=df.sample(frac=0.95, random_state=200)
df_eval=df.drop(df_train.index)
len(df_train), len(df_eval)

(2093, 110)

In [8]:
weave.init("mistral_hackathon")
ds_train = weave.Dataset(name="ds_train", rows=df_train)
ds_eval = weave.Dataset(name="ds_eval", rows=df_eval)

Logged in as Weights & Biases user: aastroza.
View Weave data at https://wandb.ai/aastroza/mistral_hackathon/weave


In [9]:
weave.publish(ds_train)
weave.publish(ds_eval)

📦 Published to https://wandb.ai/aastroza/mistral_hackathon/weave/objects/ds_train/versions/Ub9gvxpKzTUY2BI5LuMq3Za0MBGUYgwwyqQX79s6iHo
📦 Published to https://wandb.ai/aastroza/mistral_hackathon/weave/objects/ds_eval/versions/cAmPxPzqOalWMUYag6bpPlctzvG55Gego4JHA1qIprY


ObjectRef(entity='aastroza', project='mistral_hackathon', name='ds_eval', digest='cAmPxPzqOalWMUYag6bpPlctzvG55Gego4JHA1qIprY', extra=[])